In [ ]:
from Project.Data_Cleaning import clean_data
from Project.GDP_Pop_Extraction.extraction import extract_data

df = extract_data()

#This operation takes about 1.5 minutes on an average laptop, the modified dataset
#can be read from the main directory if we are not focusing on this part
data = clean_data.CleanData(df) # Use cd.CleanData(log_level = 0) to see logging messages
#The main source of the wait time is __decimal_fix in the init function
modified = data.modified
NaN = data.NaN
NoNaN = data.NoNaN
feature_tables = data.feature_tables

#To create csvs from the results of data cleaning
#data.create_csvs()

In [ ]:
# A sample slice of the dataset
#To bypass the wait time above, comment the upper part, uncomment the lines below
#****WARNING***** The PCA and map_display functions requires the CleanData object above
import pandas as pd
modified = pd.read_csv('output/modified.csv')
modified = modified.drop('Unnamed: 0',axis = 1)

In [ ]:
print('\nBelow there is a sample from the modified dataset')
modified

In [ ]:
from Project.PCA import pca_analysis

result_pca = pca_analysis.PCA_Analysis(data)

eig_vals = result_pca.eig_vals
Nmin = pca_analysis.calnum(eig_vals)
print(f'Minimum number of eigen values for the subsace to provide enough (85%) information: {Nmin}\n')

x = result_pca.x
finalDf = result_pca.finalDf
features = result_pca.features

pcoef, finalcomp = pca_analysis.calcoefficient(x,finalDf,features)


print('The 5 different feature combinations obtained from PCA are shown below.')
print('In each combination there are 4 positive and 4 negative components.\n')
for index,i in enumerate(finalcomp):
    print(f'{index+1} - ', end =" ")
    for j in i:
        print(j,end =", ")
    print('')
    
print('\nWe will use these results to determine the most relevant features.')

In [ ]:
from Project.Map_Display import map_display
from IPython.display import SVG

features = list(modified.columns)
features.remove('Country')
features.remove('Year')
print(features)

map_display.displaymap(data,features[1], 2000)

In [ ]:
map_display.displaymap(data,features[1], 2015)

In [ ]:
#map_display.display(data,features[3], 2013)
#map_display.display(data,features[4], 2012)

In [ ]:
from Project.Map_Display.analysis_part1 import analysis

#The below function prints raw output, which is the first step of our analysis
analysis(modified)